In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

In [3]:
path = './output/'
app_active = pd.read_csv(path + 'app_actived_new.csv')
# app_active = pd.read_csv(path + 'version2/app_actived_new.csv')
train_data = pd.read_csv(path + 'train_data_basic.csv')
test_data = pd.read_csv(path + 'test_data_basic.csv')

In [3]:
cat_columns = ['city','prodName','color','carrier','ct','carrier']
train_data.drop(columns=cat_columns, inplace=True)
test_data.drop(columns=cat_columns, inplace=True)

In [4]:
app_active.columns = ['uid'] + list(app_active.columns.values[1:])

In [5]:
app_active.columns

Index(['uid', 'category1', 'category2', 'category3', 'category4', 'category5',
       'category6', 'category7', 'category8', 'category9', 'category10',
       'category11', 'category12', 'category13', 'category14', 'category15',
       'category16', 'category17', 'category18', 'category19', 'category20',
       'category21', 'category22', 'category23', 'category24', 'category25',
       'category26', 'category27', 'category28', 'category29', 'category30',
       'category31', 'category32', 'category_num1', 'category_num2',
       'category_num3', 'category_num4', 'category_num5', 'category_num6',
       'category_num8', 'fre', 'app_num'],
      dtype='object')

In [6]:
train_data = train_data.merge(app_active, how='left', on='uid', suffixes=('_left', '_right'))
test_data = test_data.merge(app_active, how='left', on='uid', suffixes=('_left', '_right'))

In [7]:
app_active = None
import gc
gc.collect()

27

In [8]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2010000 entries, 0 to 2009999
Columns: 160 entries, uid to app_num
dtypes: float64(110), int64(50)
memory usage: 2.4 GB


In [9]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 502500 entries, 0 to 502499
Columns: 159 entries, uid to app_num
dtypes: float64(110), int64(49)
memory usage: 613.4 MB


In [10]:
age_group_temp = train_data.age_group
train_data.drop(columns=['age_group'], inplace=True)
train_data['age_group'] = age_group_temp

In [11]:
train_data.to_csv('./output/version3/train_data.csv', index=False)
test_data.to_csv('./output/version3/test_data.csv', index=False)

In [12]:
train_data.head()

,uid,gender,ramCapacity,ramLeftRation,romCapacity,romLeftRation,fontSize,os,bootTimes,AFuncTimes,...,category_num1,category_num2,category_num3,category_num4,category_num5,category_num6,category_num8,fre,app_num,age_group
0,1000001,0,3.0,0.43,32.0,0.46,1.15,8.0,108,0.0,...,9,9,2,1,0,0,0,571.804391,21,4
1,1000011,0,0.0,0.00,0.0,0.00,0.00,8.1,0,0.0,...,8,6,0,2,0,0,0,2741.710736,16,3
2,1000015,1,3.0,0.34,32.0,0.06,1.30,8.0,12,0.0,...,9,6,3,1,0,0,0,18991.208344,19,5
3,1000019,0,2.0,0.00,17.0,0.00,0.00,8.1,0,0.0,...,1,1,0,0,0,0,0,2085.623788,2,3
4,1000023,1,2.0,0.34,16.0,0.06,1.00,7.0,5,0.0,...,8,3,1,1,0,0,0,481.450498,13,2


In [13]:
test_data.head()

,uid,gender,ramCapacity,ramLeftRation,romCapacity,romLeftRation,fontSize,os,bootTimes,AFuncTimes,...,category32,category_num1,category_num2,category_num3,category_num4,category_num5,category_num6,category_num8,fre,app_num
0,1000002,1,0.0,0.26,0.0,0.59,1.0,9.0,14,0.0,...,6,30,31,5,3,0,0,0,11648.655418,69
1,1000003,0,8.0,0.35,128.0,0.60,1.0,9.0,13,1.0,...,5,45,27,6,2,0,0,0,19986.786189,80
2,1000004,1,3.0,0.36,32.0,0.09,1.0,8.0,57,0.0,...,0,11,10,3,2,0,0,0,3554.099370,26
3,1000005,0,6.0,0.00,68.0,0.00,0.0,9.0,0,0.0,...,2,23,8,2,1,0,0,0,1847.440280,34
4,1000006,0,2.0,0.19,16.0,0.42,0.0,6.0,33,0.0,...,0,7,1,1,1,0,0,0,323.682487,10







**-------------------------------------------------------------------------------------------------------------------------------------**

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb

from sklearn.feature_selection import SelectFromModel, VarianceThreshold, SelectKBest, chi2, mutual_info_classif, f_classif
from sklearn.preprocessing import Imputer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold

# 第一次结果

## 花费了4个小时

In [2]:
df_train = pd.read_csv('./output/version1/train_data.csv')
df_test = pd.read_csv('./output/version1/test_data.csv')
print(df_train.shape)
print(df_test.shape)

(2010000, 74)
(502500, 73)


In [7]:
print("训练模型：")
param = {
        'learning_rate': 0.1,
        'lambda_l1': 0.1,
        'lambda_l2': 0.2,
        'max_depth': 20,
        'objective': 'multiclass',
        'num_class': 7,
        'num_leaves': 31,
        'min_data_in_leaf': 50,
        'max_bin': 230,
        'feature_fraction': 0.8,
        'metric': 'multi_error',
        'nthread': 2,
        }

X = df_train.drop(['age_group','uid'], axis=1)
y = df_train['age_group']
uid = df_test['uid']
test = df_test.drop('uid', axis=1)

xx_score = []
cv_pred = []
skf = StratifiedKFold(n_splits=5, random_state=1030, shuffle=True)
for index, (train_index, vali_index) in enumerate(skf.split(X, y)):
    print(index)
    x_train, y_train, x_vali, y_vali = np.array(X)[train_index], np.array(y)[train_index], np.array(X)[vali_index], np.array(y)[vali_index]
    train = lgb.Dataset(x_train, y_train)
    vali =lgb.Dataset(x_vali, y_vali)
    print("training start...")
    model = lgb.train(param, train, num_boost_round=1000, valid_sets=[vali], early_stopping_rounds=50,verbose_eval=20)
    xx_pred = model.predict(x_vali,num_iteration=model.best_iteration)
    xx_pred = [np.argmax(x) for x in xx_pred]
    xx_score.append(f1_score(y_vali,xx_pred,average='weighted'))
    y_test = model.predict(test,num_iteration=model.best_iteration)
    y_test = [np.argmax(x) for x in y_test]
    if index == 0:
        cv_pred = np.array(y_test).reshape(-1, 1)
    else:
        cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))
        
submit = []
for line in cv_pred:
    submit.append(np.argmax(np.bincount(line)))
df = pd.DataFrame({'id':uid.as_matrix(),'label':submit})
df.to_csv('submission.csv',index=False)

训练模型：
0
training start...
Training until validation scores don't improve for 50 rounds.
[20]	valid_0's multi_error: 0.5605
[40]	valid_0's multi_error: 0.537453
[60]	valid_0's multi_error: 0.525266
[80]	valid_0's multi_error: 0.51691
[100]	valid_0's multi_error: 0.511356
[120]	valid_0's multi_error: 0.506928
[140]	valid_0's multi_error: 0.503428
[160]	valid_0's multi_error: 0.500612
[180]	valid_0's multi_error: 0.49857
[200]	valid_0's multi_error: 0.496659
[220]	valid_0's multi_error: 0.494861
[240]	valid_0's multi_error: 0.493398
[260]	valid_0's multi_error: 0.491893
[280]	valid_0's multi_error: 0.490572
[300]	valid_0's multi_error: 0.489515
[320]	valid_0's multi_error: 0.488637
[340]	valid_0's multi_error: 0.487784
[360]	valid_0's multi_error: 0.486915
[380]	valid_0's multi_error: 0.486224
[400]	valid_0's multi_error: 0.485716
[420]	valid_0's multi_error: 0.485132
[440]	valid_0's multi_error: 0.484627
[460]	valid_0's multi_error: 0.484142
[480]	valid_0's multi_error: 0.483485
[500]	va

4
training start...
Training until validation scores don't improve for 50 rounds.
[20]	valid_0's multi_error: 0.559846
[40]	valid_0's multi_error: 0.537072
[60]	valid_0's multi_error: 0.524813
[80]	valid_0's multi_error: 0.51652
[100]	valid_0's multi_error: 0.510547
[120]	valid_0's multi_error: 0.505784
[140]	valid_0's multi_error: 0.502303
[160]	valid_0's multi_error: 0.499391
[180]	valid_0's multi_error: 0.497398
[200]	valid_0's multi_error: 0.49545
[220]	valid_0's multi_error: 0.493993
[240]	valid_0's multi_error: 0.492682
[260]	valid_0's multi_error: 0.491363
[280]	valid_0's multi_error: 0.490299
[300]	valid_0's multi_error: 0.48909
[320]	valid_0's multi_error: 0.488157
[340]	valid_0's multi_error: 0.487396
[360]	valid_0's multi_error: 0.486622
[380]	valid_0's multi_error: 0.486027
[400]	valid_0's multi_error: 0.485276
[420]	valid_0's multi_error: 0.484724
[440]	valid_0's multi_error: 0.484211
[460]	valid_0's multi_error: 0.483816
[480]	valid_0's multi_error: 0.483299
[500]	valid_0

In [8]:
xx_score

[0.5171289374676408,
 0.5173886140430205,
 0.5179469808270148,
 0.5175517510172625,
 0.5174211000680055]

# 第二次结果

## 花费了8个小时

In [2]:
df_train = pd.read_csv('./output/version2/train_data.csv')
df_test = pd.read_csv('./output/version2/test_data.csv')
print(df_train.shape)
print(df_test.shape)

(2010000, 234)
(502500, 233)


In [3]:
def train_func(df_train, df_test):
    print("训练模型：")
    param = {
            'learning_rate': 0.1,
            'lambda_l1': 0.1,
            'lambda_l2': 0.2,
            'max_depth': 20,
            'objective': 'multiclass',
            'num_class': 7,
            'num_leaves': 31,
            'min_data_in_leaf': 50,
            'max_bin': 230,
            'feature_fraction': 0.8,
            'metric': 'multi_error',
            'nthread': 2,
            }

    X = df_train.drop(['age_group','uid'], axis=1)
    y = df_train['age_group']
    uid = df_test['uid']
    test = df_test.drop('uid', axis=1)

    xx_score = []
    cv_pred = []
    skf = StratifiedKFold(n_splits=5, random_state=1030, shuffle=True)
    for index, (train_index, vali_index) in enumerate(skf.split(X, y)):
        print(index)
        x_train, y_train, x_vali, y_vali = np.array(X)[train_index], np.array(y)[train_index], np.array(X)[vali_index], np.array(y)[vali_index]
        train = lgb.Dataset(x_train, y_train)
        vali =lgb.Dataset(x_vali, y_vali)
        print("training start...")
        model = lgb.train(param, train, num_boost_round=1000, valid_sets=[vali], early_stopping_rounds=50,verbose_eval=20)
        xx_pred = model.predict(x_vali,num_iteration=model.best_iteration)
        xx_pred = [np.argmax(x) for x in xx_pred]
        xx_score.append(f1_score(y_vali,xx_pred,average='weighted'))
        y_test = model.predict(test,num_iteration=model.best_iteration)
        y_test = [np.argmax(x) for x in y_test]
        if index == 0:
            cv_pred = np.array(y_test).reshape(-1, 1)
        else:
            cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))

    submit = []
    for line in cv_pred:
        submit.append(np.argmax(np.bincount(line)))
    df = pd.DataFrame({'id':uid.as_matrix(),'label':submit})
    df.to_csv('./output/version2/submission.csv',index=False)

In [4]:
train_func(df_train, df_test)

训练模型：
0
training start...
Training until validation scores don't improve for 50 rounds.
[20]	valid_0's multi_error: 0.558647
[40]	valid_0's multi_error: 0.534876
[60]	valid_0's multi_error: 0.52209
[80]	valid_0's multi_error: 0.512734
[100]	valid_0's multi_error: 0.506279
[120]	valid_0's multi_error: 0.501953
[140]	valid_0's multi_error: 0.498368
[160]	valid_0's multi_error: 0.495535
[180]	valid_0's multi_error: 0.492985
[200]	valid_0's multi_error: 0.490896
[220]	valid_0's multi_error: 0.488908
[240]	valid_0's multi_error: 0.486883
[260]	valid_0's multi_error: 0.485888
[280]	valid_0's multi_error: 0.484692
[300]	valid_0's multi_error: 0.483592
[320]	valid_0's multi_error: 0.482682
[340]	valid_0's multi_error: 0.481557
[360]	valid_0's multi_error: 0.480607
[380]	valid_0's multi_error: 0.479784
[400]	valid_0's multi_error: 0.47901
[420]	valid_0's multi_error: 0.47841
[440]	valid_0's multi_error: 0.47794
[460]	valid_0's multi_error: 0.477291
[480]	valid_0's multi_error: 0.476637
[500]	va

4
training start...
Training until validation scores don't improve for 50 rounds.
[20]	valid_0's multi_error: 0.558122
[40]	valid_0's multi_error: 0.5339
[60]	valid_0's multi_error: 0.521323
[80]	valid_0's multi_error: 0.512714
[100]	valid_0's multi_error: 0.506883
[120]	valid_0's multi_error: 0.502167
[140]	valid_0's multi_error: 0.498701
[160]	valid_0's multi_error: 0.496002
[180]	valid_0's multi_error: 0.493617
[200]	valid_0's multi_error: 0.49154
[220]	valid_0's multi_error: 0.489781
[240]	valid_0's multi_error: 0.488002
[260]	valid_0's multi_error: 0.48658
[280]	valid_0's multi_error: 0.485149
[300]	valid_0's multi_error: 0.484199
[320]	valid_0's multi_error: 0.483159
[340]	valid_0's multi_error: 0.482015
[360]	valid_0's multi_error: 0.481269
[380]	valid_0's multi_error: 0.480614
[400]	valid_0's multi_error: 0.479791
[420]	valid_0's multi_error: 0.479455
[440]	valid_0's multi_error: 0.47898
[460]	valid_0's multi_error: 0.478542
[480]	valid_0's multi_error: 0.477928
[500]	valid_0's

# 第三次结果

In [6]:
def train_func(df_train, df_test):
    print("训练模型：")
    param = {
            'learning_rate': 0.1,
            'lambda_l1': 0.1,
            'lambda_l2': 0.2,
            'max_depth': 20,
            'objective': 'multiclass',
            'num_class': 7,
            'num_leaves': 63,
            'feature_fraction': 0.8,
            'metric': 'multi_error',
            'nthread': 2,
            }

    X = df_train.drop(['age_group','uid'], axis=1)
    y = df_train['age_group']
    uid = df_test['uid']
    test = df_test.drop('uid', axis=1)

    xx_score = []
    cv_pred = []
    skf = StratifiedKFold(n_splits=5, random_state=1030, shuffle=True)
    for index, (train_index, vali_index) in enumerate(skf.split(X, y)):
        print(index)
        x_train, y_train, x_vali, y_vali = np.array(X)[train_index], np.array(y)[train_index], np.array(X)[vali_index], np.array(y)[vali_index]
        train = lgb.Dataset(x_train, y_train)
        vali =lgb.Dataset(x_vali, y_vali)
        print("training start...")
        model = lgb.train(param, train, num_boost_round=1000, valid_sets=[vali], early_stopping_rounds=50,verbose_eval=20)
        xx_pred = model.predict(x_vali,num_iteration=model.best_iteration)
        xx_pred = [np.argmax(x) for x in xx_pred]
        xx_score.append(f1_score(y_vali,xx_pred,average='weighted'))
        y_test = model.predict(test,num_iteration=model.best_iteration)
        y_test = [np.argmax(x) for x in y_test]
        if index == 0:
            cv_pred = np.array(y_test).reshape(-1, 1)
        else:
            cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))
        print('XX_score is ', xx_score)
    submit = []
    for line in cv_pred:
        submit.append(np.argmax(np.bincount(line)))
    df = pd.DataFrame({'id':uid.as_matrix(),'label':submit})
    df.to_csv('./output/version2/submission2.csv',index=False)

In [ ]:
train_func(df_train, df_test)

训练模型：
0
training start...
Training until validation scores don't improve for 50 rounds.
[20]	valid_0's multi_error: 0.548383
[40]	valid_0's multi_error: 0.525634
[60]	valid_0's multi_error: 0.513122
[80]	valid_0's multi_error: 0.504771
[100]	valid_0's multi_error: 0.499097
[120]	valid_0's multi_error: 0.494659
[140]	valid_0's multi_error: 0.491512
[160]	valid_0's multi_error: 0.488555
[180]	valid_0's multi_error: 0.486231
[200]	valid_0's multi_error: 0.48402
[220]	valid_0's multi_error: 0.482689
[240]	valid_0's multi_error: 0.481453
[260]	valid_0's multi_error: 0.479985
[280]	valid_0's multi_error: 0.478729
[300]	valid_0's multi_error: 0.477425
[320]	valid_0's multi_error: 0.476609
[340]	valid_0's multi_error: 0.475803
[360]	valid_0's multi_error: 0.475109
[380]	valid_0's multi_error: 0.474555
[400]	valid_0's multi_error: 0.47406
[420]	valid_0's multi_error: 0.473527
[440]	valid_0's multi_error: 0.473047
[460]	valid_0's multi_error: 0.472806
[480]	valid_0's multi_error: 0.472254
[500]	

# 第四次结果

In [14]:
def train_func(df_train, df_test):
    print("训练模型：")
    param = {
            'learning_rate': 0.1,
            'lambda_l1': 0.1,
            'lambda_l2': 0.2,
            'max_depth': 20,
            'objective': 'multiclass',
            'num_class': 7,
            'num_leaves': 31,
            'min_data_in_leaf': 50,
            'max_bin': 230,
            'feature_fraction': 0.5,
            'metric': 'multi_error',
            'nthread': 2,
            }

    X = df_train.drop(['age_group','uid'], axis=1)
    y = df_train['age_group']
    uid = df_test['uid']
    test = df_test.drop('uid', axis=1)

    xx_score = []
    cv_pred = []
    skf = StratifiedKFold(n_splits=5, random_state=1030, shuffle=True)
    for index, (train_index, vali_index) in enumerate(skf.split(X, y)):
        print(index)
        x_train, y_train, x_vali, y_vali = np.array(X)[train_index], np.array(y)[train_index], np.array(X)[vali_index], np.array(y)[vali_index]
        train = lgb.Dataset(x_train, y_train)
        vali =lgb.Dataset(x_vali, y_vali)
        print("training start...")
        model = lgb.train(param, train, num_boost_round=2000, valid_sets=[vali], early_stopping_rounds=50,verbose_eval=20)
        xx_pred = model.predict(x_vali,num_iteration=model.best_iteration)
        xx_pred = [np.argmax(x) for x in xx_pred]
        xx_score.append(f1_score(y_vali,xx_pred,average='weighted'))
        y_test = model.predict(test,num_iteration=model.best_iteration)
        y_test = [np.argmax(x) for x in y_test]
        if index == 0:
            cv_pred = np.array(y_test).reshape(-1, 1)
        else:
            cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))

    submit = []
    for line in cv_pred:
        submit.append(np.argmax(np.bincount(line)))
    df = pd.DataFrame({'id':uid.as_matrix(),'label':submit})
    df.to_csv('./output/version3/submission.csv',index=False)

In [15]:
train_func(train_data, test_data)

训练模型：
0
training start...
Training until validation scores don't improve for 50 rounds.
[20]	valid_0's multi_error: 0.55799
[40]	valid_0's multi_error: 0.53605
[60]	valid_0's multi_error: 0.524515
[80]	valid_0's multi_error: 0.515794
[100]	valid_0's multi_error: 0.510368
[120]	valid_0's multi_error: 0.505726
[140]	valid_0's multi_error: 0.501868
[160]	valid_0's multi_error: 0.498445
[180]	valid_0's multi_error: 0.495794
[200]	valid_0's multi_error: 0.493577
[220]	valid_0's multi_error: 0.491614
[240]	valid_0's multi_error: 0.489968
[260]	valid_0's multi_error: 0.488405
[280]	valid_0's multi_error: 0.487174
[300]	valid_0's multi_error: 0.485928
[320]	valid_0's multi_error: 0.484709
[340]	valid_0's multi_error: 0.483811
[360]	valid_0's multi_error: 0.482799
[380]	valid_0's multi_error: 0.48191
[400]	valid_0's multi_error: 0.481164
[420]	valid_0's multi_error: 0.480545
[440]	valid_0's multi_error: 0.480065
[460]	valid_0's multi_error: 0.479585
[480]	valid_0's multi_error: 0.478935
[500]	v

[1140]	valid_0's multi_error: 0.47349
Early stopping, best iteration is:
[1101]	valid_0's multi_error: 0.473386
3
training start...
Training until validation scores don't improve for 50 rounds.
[20]	valid_0's multi_error: 0.559495
[40]	valid_0's multi_error: 0.535373
[60]	valid_0's multi_error: 0.523443
[80]	valid_0's multi_error: 0.514801
[100]	valid_0's multi_error: 0.509351
[120]	valid_0's multi_error: 0.504333
[140]	valid_0's multi_error: 0.501
[160]	valid_0's multi_error: 0.498045
[180]	valid_0's multi_error: 0.495779
[200]	valid_0's multi_error: 0.493667
[220]	valid_0's multi_error: 0.492075
[240]	valid_0's multi_error: 0.490488
[260]	valid_0's multi_error: 0.489289
[280]	valid_0's multi_error: 0.487908
[300]	valid_0's multi_error: 0.48656
[320]	valid_0's multi_error: 0.4855
[340]	valid_0's multi_error: 0.484699
[360]	valid_0's multi_error: 0.483644
[380]	valid_0's multi_error: 0.482905
[400]	valid_0's multi_error: 0.482189
[420]	valid_0's multi_error: 0.481622
[440]	valid_0's mu

In [ ]:
#先尝试玄学调参，哈哈哈

# 第五次结果

In [3]:
df_train = pd.read_csv('./output/version4/train_data.csv')
df_test = pd.read_csv('./output/version4/test_data.csv')
print(df_train.shape)
print(df_test.shape)

(2010000, 85)
(502500, 84)


In [4]:
def train_func(df_train, df_test):
    print("训练模型：")
    param = {
            'learning_rate': 0.1,
            'lambda_l1': 0.1,
            'lambda_l2': 0.2,
            'max_depth': 20,
            'objective': 'multiclass',
            'num_class': 7,
            'num_leaves': 31,
            'min_data_in_leaf': 50,
            'max_bin': 230,
            'feature_fraction': 0.8,
            'metric': 'multi_error',
            'nthread': 2,
            }

    X = df_train.drop(['age_group','uid'], axis=1)
    y = df_train['age_group']
    uid = df_test['uid']
    test = df_test.drop('uid', axis=1)

    xx_score = []
    cv_pred = []
    skf = StratifiedKFold(n_splits=5, random_state=1030, shuffle=True)
    for index, (train_index, vali_index) in enumerate(skf.split(X, y)):
        print(index)
        x_train, y_train, x_vali, y_vali = np.array(X)[train_index], np.array(y)[train_index], np.array(X)[vali_index], np.array(y)[vali_index]
        train = lgb.Dataset(x_train, y_train)
        vali =lgb.Dataset(x_vali, y_vali)
        print("training start...")
        model = lgb.train(param, train, num_boost_round=2000, valid_sets=[vali], early_stopping_rounds=500,verbose_eval=50)
        xx_pred = model.predict(x_vali,num_iteration=model.best_iteration)
        xx_pred = [np.argmax(x) for x in xx_pred]
        xx_score.append(f1_score(y_vali,xx_pred,average='weighted'))
        y_test = model.predict(test,num_iteration=model.best_iteration)
        y_test = [np.argmax(x) for x in y_test]
        if index == 0:
            cv_pred = np.array(y_test).reshape(-1, 1)
        else:
            cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))
        print('XX_score is ', xx_score)
    submit = []
    for line in cv_pred:
        submit.append(np.argmax(np.bincount(line)))
    df = pd.DataFrame({'id':uid.as_matrix(),'label':submit})
    df.to_csv('submission.csv',index=False)

In [5]:
train_func(df_train, df_test)

训练模型：
0
training start...
Training until validation scores don't improve for 500 rounds.
[50]	valid_0's multi_error: 0.467948
[100]	valid_0's multi_error: 0.454418
[150]	valid_0's multi_error: 0.44758
[200]	valid_0's multi_error: 0.443413
[250]	valid_0's multi_error: 0.440453
[300]	valid_0's multi_error: 0.438361
[350]	valid_0's multi_error: 0.436614
[400]	valid_0's multi_error: 0.43549
[450]	valid_0's multi_error: 0.434699
[500]	valid_0's multi_error: 0.433771
[550]	valid_0's multi_error: 0.433139
[600]	valid_0's multi_error: 0.432761
[650]	valid_0's multi_error: 0.432122
[700]	valid_0's multi_error: 0.431891
[750]	valid_0's multi_error: 0.431699
[800]	valid_0's multi_error: 0.43141
[850]	valid_0's multi_error: 0.431221
[900]	valid_0's multi_error: 0.43098
[950]	valid_0's multi_error: 0.430796
[1000]	valid_0's multi_error: 0.43043
[1050]	valid_0's multi_error: 0.430174
[1100]	valid_0's multi_error: 0.429918
[1150]	valid_0's multi_error: 0.429851
[1200]	valid_0's multi_error: 0.429734


[1450]	valid_0's multi_error: 0.430746
[1500]	valid_0's multi_error: 0.43058
[1550]	valid_0's multi_error: 0.43042
[1600]	valid_0's multi_error: 0.430323
[1650]	valid_0's multi_error: 0.430127
[1700]	valid_0's multi_error: 0.43007
[1750]	valid_0's multi_error: 0.429975
[1800]	valid_0's multi_error: 0.4299
[1850]	valid_0's multi_error: 0.429978
[1900]	valid_0's multi_error: 0.429667
[1950]	valid_0's multi_error: 0.429565
[2000]	valid_0's multi_error: 0.429595
Did not meet early stopping. Best iteration is:
[1979]	valid_0's multi_error: 0.429473
XX_score is  [0.5694869660018405, 0.5679489247704582, 0.5688499329952923, 0.5674219949326158, 0.5678589657542582]


- 总共花费了5+7个小时
0.57

# 第五次模型训练

In [2]:
df_train = pd.read_csv('./output/version5/train_data.csv')
df_test = pd.read_csv('./output/version5/test_data.csv')
print(df_train.shape)
print(df_test.shape)

(2010000, 172)
(502500, 171)


In [3]:
def train_func(df_train, df_test):
    print("训练模型：")
    param = {
            'learning_rate': 0.25,
            'lambda_l1': 0.1,
            'lambda_l2': 0.2,
            'max_depth': 20,
            'objective': 'multiclass',
            'num_class': 7,
            'num_leaves': 31,
            'min_data_in_leaf': 50,
            'max_bin': 230,
            'feature_fraction': 0.8,
            'metric': 'multi_error',
            'nthread': 2,
            }

    X = df_train.drop(['age_group','uid'], axis=1)
    y = df_train['age_group']
    uid = df_test['uid']
    test = df_test.drop('uid', axis=1)

    xx_score = []
    cv_pred = []
    skf = StratifiedKFold(n_splits=5, random_state=1030, shuffle=True)
    for index, (train_index, vali_index) in enumerate(skf.split(X, y)):
        print(index)
        x_train, y_train, x_vali, y_vali = np.array(X)[train_index], np.array(y)[train_index], np.array(X)[vali_index], np.array(y)[vali_index]
        train = lgb.Dataset(x_train, y_train)
        vali =lgb.Dataset(x_vali, y_vali)
        print("training start...")
        model = lgb.train(param, train, num_boost_round=2000, valid_sets=[vali], early_stopping_rounds=500,verbose_eval=50)
        xx_pred = model.predict(x_vali,num_iteration=model.best_iteration)
        xx_pred = [np.argmax(x) for x in xx_pred]
        xx_score.append(f1_score(y_vali,xx_pred,average='weighted'))
        y_test = model.predict(test,num_iteration=model.best_iteration)
        y_test = [np.argmax(x) for x in y_test]
        if index == 0:
            cv_pred = np.array(y_test).reshape(-1, 1)
        else:
            cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))
        print('XX_score is ', xx_score)
    submit = []
    for line in cv_pred:
        submit.append(np.argmax(np.bincount(line)))
    df = pd.DataFrame({'id':uid.as_matrix(),'label':submit})
    df.to_csv('submission.csv',index=False)

In [ ]:
train_func(df_train, df_test)

训练模型：
0
training start...
Training until validation scores don't improve for 500 rounds.
[50]	valid_0's multi_error: 0.451256
[100]	valid_0's multi_error: 0.440567
[150]	valid_0's multi_error: 0.43595
[200]	valid_0's multi_error: 0.433741
[250]	valid_0's multi_error: 0.432639
[300]	valid_0's multi_error: 0.431903
[350]	valid_0's multi_error: 0.431209
[400]	valid_0's multi_error: 0.430878
[450]	valid_0's multi_error: 0.4305
[500]	valid_0's multi_error: 0.429995
[550]	valid_0's multi_error: 0.429826
[600]	valid_0's multi_error: 0.429308
[650]	valid_0's multi_error: 0.429336
[700]	valid_0's multi_error: 0.429007
[750]	valid_0's multi_error: 0.428905
[800]	valid_0's multi_error: 0.428925
[850]	valid_0's multi_error: 0.428468
[900]	valid_0's multi_error: 0.428567
[950]	valid_0's multi_error: 0.42852
[1000]	valid_0's multi_error: 0.428239
[1050]	valid_0's multi_error: 0.428142
[1100]	valid_0's multi_error: 0.428408
[1150]	valid_0's multi_error: 0.428323
[1200]	valid_0's multi_error: 0.428147